<a href="https://colab.research.google.com/github/JulizarHw31/MERA/blob/circleci-project-setup/Machine_Learning_LSTM_MERA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MOUNT DATA GOOGLE DRIVE


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#For Indonesia Language



In [3]:
pip install Sastrawi

     |████████████████████████████████| 215kB 3.1MB/s 


## Importing Libraries 

In [4]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk
import numpy as np
import tensorflow as tf
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#Importing the dataset

In [5]:
df = pd.read_csv('/content/drive/MyDrive/dataset/Dataset02.csv')
df.head()

,Komentar,Skor
0,Saya sering diejek karena memiliki ukuran pant...,0
1,teman sma saya dulu sering mengejek saya bahwa...,0
2,Saya pernah mengalami tindakan tidak menyenang...,0
3,"teman saya memiliki kulit yang cukup gelap, su...",0
4,Cat Calling sering terjadi saat saya berjalan ...,0


#Split Some Parahraphs Into Several Sentence 

In [6]:
#import stopword
from nltk.tokenize import sent_tokenize, word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

#Text preprocessing



In [7]:
def pre_process(text):
    # Case Folding: Lowercase
    text = text.lower()
    
    #Separating Sentences with Split () Method
    pisah = text.split()

    # Case Folding: Removing Number
    text = re.sub(r"\d+", "", text)

    # Case Folding: Removing Punctuation
    text = text.translate(str.maketrans("","",string.punctuation))

    #Case Folding: Removing Whitespace
    text = text.strip()

    #Filtering using sastrawi
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    text =  stopword.remove(text)

    return text

df['Komentar'] = df['Komentar'].apply(lambda x:pre_process(x))
df.head()

,Komentar,Skor
0,sering diejek memiliki ukuran pantat mencolok ...,0
1,teman sma dulu sering mengejek bahwa gendut pe...,0
2,pernah mengalami tindakan menyenangkan dilakuk...,0
3,teman memiliki kulit cukup gelap suatu jajan k...,0
4,cat calling sering terjadi saya berjalan sekit...,0


#BUILD MODEL

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42)
print("Training data size : ", train_df.shape)
print("Test data size : ", test_df.shape)

Training data size :  (59, 2)
Test data size :  (15, 2)


#Tokenizer

In [10]:
top_words = 10000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(train_df['Komentar'])
list_tokenized_train = tokenizer.texts_to_sequences(train_df['Komentar'])

max_review_length = 200
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = train_df['Skor']

#MAKE MODEL SEQUENTIAL

In [11]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           320032    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 373,333
Trainable params: 373,333
Non-trainable params: 0
_________________________________________________________________


#Train Model

In [39]:
model.fit(X_train,y_train, epochs=10, batch_size=64, validation_split=0.2)


Epoch 1/10
1/1 [==============================] - 0s 271ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.7792 - val_accuracy: 0.5833
Epoch 2/10
1/1 [==============================] - 0s 236ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.7800 - val_accuracy: 0.5833
Epoch 3/10
1/1 [==============================] - 0s 256ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.7809 - val_accuracy: 0.5833
Epoch 4/10
1/1 [==============================] - 0s 248ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.7819 - val_accuracy: 0.5833
Epoch 5/10
1/1 [==============================] - 0s 236ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.7828 - val_accuracy: 0.5833
Epoch 6/10
1/1 [==============================] - 0s 248ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.7839 - val_accuracy: 0.5833
Epoch 7/10
1/1 [==============================] - 0s 244ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.7849 - val_accuracy: 0.5833
Epoch 8/10
1/1 [====

In [40]:
list_tokenized_test = tokenizer.texts_to_sequences(test_df['Komentar'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = test_df['Skor']
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  1.0
F1-score:  1.0
Confusion matrix:


array([[7, 0],
       [0, 8]])

#EXPORT MODEL

In [14]:
export_dir = 'saved_model/1'

tf.saved_model.save(model, export_dir)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: saved_model/1/assets


INFO:tensorflow:Assets written to: saved_model/1/assets


In [15]:
# Select mode of optimization
mode = "Speed" 

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

#SAVE MODEL TFLITE

In [20]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimzations
converter.optimizations =  [optimization]
# Invoke the converter to finally generate the TFLite model
tflite_model =converter.convert()

In [ ]:
import pathlib
tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

467648